In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt

rf_rate = 0.02

In [48]:
# data: 
xls_dict  = pd.read_excel('data/trading-game-data-08112023.xlsx', sheet_name=None)
#data = pd.read_excel('trading-game-data-20102023.xlsx', sheet_name='price')
info_df = xls_dict['info'][['Ticker', 'RBICS Economy']]
index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

info_df = info_df.set_index('Ticker')

## ESG ratings
We will be using the esg ratings from https://www.gigasheet.com/sample-data/sp-500-esg-risk-ratings. Some companies are present in the dataframe therefore we will be using the sector median (and merged based on the info tab in the Trading game datasheet).

In [69]:
esgratings_df = pd.read_csv('data\esg data.csv')
esgratings_df = esgratings_df[['Symbol', 'Sector', 'Total ESG Risk score']]
sector_medians = esgratings_df.groupby(esgratings_df['Sector']).median()

esgratings_df.set_index('Symbol', inplace=True)
esgratings_df = esgratings_df.T

full_esg_df = pd.DataFrame(index=info_df.index)
full_esg_df['ESG Score'] = np.nan  # Initialize all scores as NaN

# Iterate over all companies in the S&P 500
for company in full_esg_df.index:
    if company in esgratings_df.index:
        # Use the actual ESG score if available
        full_esg_df.loc[company, 'ESG Score'] = esgratings_df.loc[company, 'Total ESG Risk score']
    else:
        # Retrieve the sector from info_df
        sector = info_df.loc[company, 'RBICS Economy']
        # Use the sector median if the sector is available
        if sector in sector_medians.index:
            full_esg_df.loc[company, 'ESG Score'] = sector_medians.loc[sector, 'Total ESG Risk score']

# Handling cases where the sector is unknown or no median is available
full_esg_df['ESG Score'].fillna(full_esg_df['ESG Score'].median(), inplace=True)
full_esg_df['Inverted_ESG'] = -full_esg_df['ESG Score']
full_esg_df

,ESG Score,Inverted_ESG
Ticker,,
A,22.0,-22.0
AAL,23.0,-23.0
AAPL,16.0,-16.0
ABBV,22.0,-22.0
ABNB,22.0,-22.0
...,...,...
YUM,22.0,-22.0
ZBH,22.0,-22.0
ZBRA,16.0,-16.0


In [4]:
price_df = xls_dict['price'].reset_index()
price_df = price_df.drop('index', axis = 1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.set_index('Date', inplace=True)
daily_returns = price_df.pct_change()
daily_returns = daily_returns.drop(daily_returns.index[0])
expected_returns = daily_returns.mean()
risk = daily_returns.std()
average_returns = daily_returns.mean() * len(daily_returns)
average_returns 

A      -0.274701
AAL    -0.024674
AAPL    0.355328
ABBV   -0.110380
ABNB    0.427907
          ...   
YUM    -0.009085
ZBH    -0.165946
ZBRA   -0.134531
ZION   -0.178025
ZTS     0.159445
Length: 500, dtype: float64

In [66]:
#Calculating the factors per date, SMB
#SMB needs average returns of stocks
size_df = xls_dict['size'].reset_index()
size_df = size_df.drop('index', axis = 1)
size_df['Date'] = pd.to_datetime(size_df['Date'])
size_df.set_index('Date', inplace=True)


average_returns_top =[]
average_returns_low = []

for i in range(len(size_df)-1):
    top_stocks = size_df.iloc[i].nlargest(int(len(size_df.iloc[i])*0.3)).index
    low_stocks = size_df.iloc[i].nsmallest(int(len(size_df.iloc[i])*0.3)).index
    average_returns_top.append(daily_returns.iloc[i][top_stocks].mean())
    average_returns_low.append(daily_returns.iloc[i][low_stocks].mean())

ave_return_df = pd.DataFrame({'top': average_returns_top, 'low': average_returns_low})
smb = ave_return_df['low'] - ave_return_df['top']

smb

0      0.000175
1      0.012152
2      0.000187
3      0.000964
4      0.006410
         ...   
209   -0.011296
210    0.006039
211    0.018426
212   -0.011327
213   -0.000621
Length: 214, dtype: float64

In [19]:
# Now HML
#calculate market to book. As price to book / price = 1/book value bepalen
#make new dataframe with price divided by price to book
price_to_book_df = xls_dict['price-to-book'].reset_index()
price_to_book_df = price_to_book_df.drop('index', axis = 1)
price_to_book_df['Date'] = pd.to_datetime(price_to_book_df['Date'])
price_to_book_df.set_index('Date', inplace=True)

bookvalue_df = price_df / price_to_book_df
btm_df = bookvalue_df / size_df
btm_df

def top_low_stocks(row):
    top_stocks = row.nlargest(int(len(row)*0.3)).index
    low_stocks = row.nsmallest(int(len(row)*0.3)).index
    average_returns_top = average_returns[top_stocks].mean()
    average_returns_low = average_returns[low_stocks].mean()
    #print(average_returns_low)

    return average_returns_top, average_returns_low

ave_return_df_hml = btm_df.apply(top_low_stocks, axis = 1, result_type = 'expand')
hml = ave_return_df_hml[1] - ave_return_df_hml[0]
hml
#now Mkt-rf
#Calculate market returns
index_price_df = xls_dict['index-price'].reset_index()
index_price_df = index_price_df.drop('index', axis = 1)
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
daily_market_returns = index_price_df.pct_change()
mkt_rf = daily_market_returns - rf_rate
mkt_rf


,S&P 500
Date,
2022-12-30,NaN
2023-01-03,-0.024001
2023-01-04,-0.012461
2023-01-05,-0.031646
2023-01-06,0.002841
...,...
2023-11-01,-0.009494
2023-11-02,-0.001141
2023-11-03,-0.010606


In [18]:
#now Mkt-rf
#Calculate risk free rate

#Calculate market returns
index_price_df = xls_dict['index-price'].reset_index()
index_price_df = index_price_df.drop('index', axis = 1)
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
daily_market_returns = index_price_df.pct_change()
mkt_rf = daily_market_returns - rf_rate
mkt_rf

,S&P 500
Date,
2022-12-30,NaN
2023-01-03,-0.024001
2023-01-04,-0.012461
2023-01-05,-0.031646
2023-01-06,0.002841
...,...
2023-11-01,-0.009494
2023-11-02,-0.001141
2023-11-03,-0.010606


In [14]:
#now RMW
#calculate operating profitability
turnover_df = xls_dict['turnover'].reset_index()
turnover_df = turnover_df.drop('index', axis = 1)
turnover_df['Date'] = pd.to_datetime(turnover_df['Date'])
turnover_df.set_index('Date', inplace=True)



In [15]:
# now CMA
#calculate investment
#make new dataframe with price divided by price to book
price_to_book_df = xls_dict['price-to-book'].reset_index()
price_to_book_df = price_to_book_df.drop('index', axis = 1)
price_to_book_df['Date'] = pd.to_datetime(price_to_book_df['Date'])
price_to_book_df.set_index('Date', inplace=True)



In [16]:
price_to_book_df

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-30,8.706802,NaN,40.881115,17.868654,9.770078,5.365857,2.114528,7.618747,10.934358,2.302693,...,NaN,2.340734,2.440706,1.893832,6.197422,NaN,2.184578,5.000396,1.728119,14.659472
2023-01-03,8.729493,NaN,39.351967,17.953790,9.701515,5.355594,2.103750,7.716394,10.947030,2.280513,...,NaN,2.339399,2.356841,1.944390,6.208072,NaN,2.180808,5.074307,1.703512,14.689481
2023-01-04,8.441456,NaN,39.757854,18.098633,10.138026,5.435258,2.114191,7.690126,11.092916,2.317196,...,NaN,2.359431,2.363701,2.009223,6.198543,NaN,2.201026,5.128716,1.765733,14.900545
2023-01-05,8.465945,NaN,39.336235,18.076520,10.022614,5.415220,2.119917,7.508538,10.671501,2.230324,...,NaN,2.311688,2.416587,1.968182,6.022547,NaN,2.160248,5.071577,1.734798,14.549437
2023-01-06,8.218829,NaN,40.783577,18.414854,10.115172,5.489997,2.152252,7.686414,10.811540,2.311749,...,NaN,2.380130,2.445796,1.988405,6.236657,NaN,2.153737,5.332118,1.801941,14.768504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-01,5.417768,NaN,45.164722,19.542273,14.972116,4.435259,2.430020,7.486222,16.733524,2.210981,...,NaN,1.973529,2.124518,1.761692,2.297337,NaN,1.773319,3.396861,0.948880,15.086717
2023-11-02,5.502568,NaN,46.099327,19.650633,14.474591,4.479607,2.320399,7.534435,16.149326,2.281792,...,NaN,1.990806,2.194304,1.630308,2.293203,NaN,1.859900,3.551444,1.016482,16.030134
2023-11-03,5.742222,NaN,45.860480,19.398245,15.369385,4.474005,2.309869,7.665748,16.292404,2.331499,...,NaN,1.989502,2.167556,1.668580,2.356910,NaN,1.835428,3.676688,1.063284,16.161634


In [ ]:
def calc_esgratings()